In [7]:
import mfem.ser as mfem
import numpy as np
import pandas as pd

In [48]:
mesh = mfem.Mesh('room_heater', 1, 1)
print(mesh.PrintInfo())

Mesh Characteristics:
Dimension          : 3
Space dimension    : 3
Number of vertices : 8134
Number of edges    : 51214
Number of faces    : 82765  --  82765 Triangle(s)
Number of elements : 39684  --  39684 Tetrahedron(s)
Number of bdr elem : 6794  --  6794 Triangle(s)
Euler Number       : 1
h_min              : 70.28
h_max              : 330.49
kappa_min          : 1.05177
kappa_max          : 8.11726

None


In [49]:
print([i for i in mesh.bdr_attributes])
print([i for i in mesh.attributes])

[1, 2]
[1, 2]


In [50]:
# Создание пространства конечных элементов
fec = mfem.H1_FECollection(1, mesh.Dimension())   # H1 order=1
fespace = mfem.FiniteElementSpace(mesh, fec)

print(fespace.GetTrueVSize())

8134


In [17]:
def Solver(mesh, fespace, f):
    # Граничное условие Дирихле - на плоскости окна
    t1 = mfem.ConstantCoefficient(9.0)
    # Граничное условие Неймана - отсутствие потока тепла через стенки
    t2 = mfem.ConstantCoefficient(0.0)
    #Стенка и окно
    ess_bdr1 = mfem.intArray([0]*mesh.bdr_attributes.Size()) 
    ess_bdr1[0] = 1
    ess_bdr2 = mfem.intArray([0]*mesh.bdr_attributes.Size())
    ess_bdr2[1] = 1

    # Применение граничных условий Дирихле
    ess_tdof_list = mfem.intArray()
    ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
    ess_bdr[1] = 0
    fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

    # Задаем коэффициенты теплопроводности для объема воздуха и алюминиевого радиатора
    k1 = 203.5 #алюминий
    k2 = 0.022 #воздух
    k_values = mfem.Vector(mesh.attributes.Max())
    k_values[0] = k1
    k_values[1] = k2

    # Создаем коэффициент с кусочно-постоянными значениями
    k_pw_const = mfem.PWConstCoefficient(k_values)

    # Создаем источник тепла
    class VolumeSource(mfem.PyCoefficient):
        def __init__(self, mesh):
            super().__init__()
            self.mesh = mesh

        def Eval(self, Ttr, ip):
            element_attribute = Ttr.ElementNo

            if self.mesh.GetAttribute(element_attribute) == 1:
                return f
            elif self.mesh.GetAttribute(element_attribute) == 2:
                return 0.0
            else:
                return 0.0

    rhs = VolumeSource(mesh)


    # Создание системы уравнений
    a = mfem.BilinearForm(fespace)
    a.AddDomainIntegrator(mfem.DiffusionIntegrator(k_pw_const))
    a.Assemble()

    # Создание линейной формы для правой части
    b = mfem.LinearForm(fespace)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(rhs))
    b.AddBoundaryIntegrator(mfem.BoundaryLFIntegrator(t2), ess_bdr2)
    b.Assemble()

    # Задание граничных условий Дирихле
    x = mfem.GridFunction(fespace)
    x.Assign(0.0) # Начальное значение температуры во всей области
    x.ProjectBdrCoefficient(t1, ess_bdr1)

    # Формирование линейной системы (AX=B)
    A = mfem.OperatorPtr()
    B = mfem.Vector()
    X = mfem.Vector()
    a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)
    print("Size of linear system: " + str(A.Height()))

    AA = mfem.OperatorHandle2SparseMatrix(A)
    M = mfem.GSSmoother(AA)
    mfem.PCG(AA, M, B, X, 1, 500, 1e-12, 0.0)
    a.RecoverFEMSolution(X, b, x)

    return x

In [1]:
'''
    Методом дихотомии, находим мощность батареи ( == f ), которая подходит для 
    достижения заданной температуры в узлах сетки, приближенных к центру
'''
def find_goal_f(mesh, fespace, point, f_min, f_max, iteration_max, tol, temp_goal):
    def has_converged(current_temp, goal_temp, tolerance):
        return abs(goal_temp - current_temp) <= tolerance

    verts = mesh.GetVertexArray()
    index_nearest_center = np.argmin([np.linalg.norm(v - point) for v in verts])

    iteration = 0
    current_temp = 0

    if has_converged(current_temp, temp_goal, tol):
        return None, iteration, current_temp, None, index_nearest_center, verts[index_nearest_center]

    while iteration < iteration_max and not has_converged(current_temp, temp_goal, tol):
        iteration += 1
        current_f = (f_min + f_max) / 2
        x = Solver(mesh, fespace, current_f)
        sol = x.GetDataArray()

        current_temp = sol[index_nearest_center]

        if current_temp < temp_goal:
            f_min = current_f
        else:
            f_max = current_f

    return x, iteration, current_temp, current_f, index_nearest_center, verts[index_nearest_center]


In [18]:
def find_temp_radiator_max(mesh, sol):
    temp_radiator_max = -np.inf
    elements = mesh.GetNE()

    for i in range(elements):
        if mesh.GetAttribute(i) == 1:
            temp_radiator_max = max(temp_radiator_max, sol[i])

    return temp_radiator_max if temp_radiator_max != -np.inf else None
 
def solution(msh_file, center, radiator_position, f_min, f_max, iteration_max, tol, temp_goal):
    mesh = mfem.Mesh(msh_file, 1, 1)
    fec = mfem.H1_FECollection(1, mesh.Dimension())
    fespace = mfem.FiniteElementSpace(mesh, fec)

    x, iterations, temp_center_cur, f_cur, index_nearest_center, vert = find_goal_f(mesh, fespace, center, f_min, f_max, iteration_max, tol, temp_goal)
    sol = x.GetDataArray()
    temp_radiator_max = find_temp_radiator_max(radiator_position, mesh, sol)
    vert_str = f"({vert[0]:.0f}, {vert[1]:.0f}, {vert[2]:.0f})"

    save(mesh, x, msh_file)

    return [fespace.GetTrueVSize(), iterations, temp_center_cur, temp_radiator_max, f_cur, vert_str, np.linalg.norm(vert - center)]


In [ ]:
def save(mesh, x, filename):
    pd = mfem.ParaViewDataCollection(filename, mesh)
    pd.SetPrefixPath("ParaView")
    pd.RegisterField("temp", x)
    pd.Save()
    

In [ ]:
# Задание начальных условий

In [19]:
center = np.array([2800, 1900, 1550])
radiator_position = np.array([[0, 200, -2650], [140, 1200, -1150]])
f_min = 0.000001
f_max = 0.00001
tol = 1e-6
temp_goal = 16.
iteration_max = 50


In [20]:
results = solution('room_heater', center, radiator_position, f_min, f_max, iteration_max, tol, temp_goal)


Size of linear system: 8134
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.32
   Iteration :   2  (B r, r) = 625.199
   Iteration :   3  (B r, r) = 293.212
   Iteration :   4  (B r, r) = 191.846
   Iteration :   5  (B r, r) = 148.978
   Iteration :   6  (B r, r) = 116.145
   Iteration :   7  (B r, r) = 91.713
   Iteration :   8  (B r, r) = 71.4997
   Iteration :   9  (B r, r) = 55.0968
   Iteration :  10  (B r, r) = 42.3497
   Iteration :  11  (B r, r) = 36.2408
   Iteration :  12  (B r, r) = 32.4868
   Iteration :  13  (B r, r) = 30.4392
   Iteration :  14  (B r, r) = 26.706
   Iteration :  15  (B r, r) = 24.9628
   Iteration :  16  (B r, r) = 23.0139
   Iteration :  17  (B r, r) = 20.3412
   Iteration :  18  (B r, r) = 21.2016
   Iteration :  19  (B r, r) = 28.4793
   Iteration :  20  (B r, r) = 40.0166
   Iteration :  21  (B r, r) = 50.2082
   Iteration :  22  (B r, r) = 51.9815
   Iteration :  23  (B r, r) = 56.7212
   Iteration :  24  (B r, r) = 61.9504

Size of linear system: 8134
Average reduction factor = 0.866993
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.37
   Iteration :   2  (B r, r) = 625.263
   Iteration :   3  (B r, r) = 293.296
   Iteration :   4  (B r, r) = 191.964
   Iteration :   5  (B r, r) = 149.138
   Iteration :   6  (B r, r) = 116.339
   Iteration :   7  (B r, r) = 91.9379
   Iteration :   8  (B r, r) = 71.7475
   Iteration :   9  (B r, r) = 55.3606
   Iteration :  10  (B r, r) = 42.6259
   Iteration :  11  (B r, r) = 36.5549
   Iteration :  12  (B r, r) = 32.8483
   Iteration :  13  (B r, r) = 30.8603
   Iteration :  14  (B r, r) = 27.1558
   Iteration :  15  (B r, r) = 25.4664
   Iteration :  16  (B r, r) = 23.5613
   Iteration :  17  (B r, r) = 20.9068
   Iteration :  18  (B r, r) = 21.8803
   Iteration :  19  (B r, r) = 29.495
   Iteration :  20  (B r, r) = 41.5557
   Iteration :  21  (B r, r) = 52.2526
   Iteration :  22  (B r, r) = 54.2066
   Iteration :  23  (B r, r) = 59.265
  

Size of linear system: 8134
Average reduction factor = 0.866885
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.34
   Iteration :   2  (B r, r) = 625.222
   Iteration :   3  (B r, r) = 293.242
   Iteration :   4  (B r, r) = 191.889
   Iteration :   5  (B r, r) = 149.037
   Iteration :   6  (B r, r) = 116.216
   Iteration :   7  (B r, r) = 91.7954
   Iteration :   8  (B r, r) = 71.5905
   Iteration :   9  (B r, r) = 55.1935
   Iteration :  10  (B r, r) = 42.451
   Iteration :  11  (B r, r) = 36.3559
   Iteration :  12  (B r, r) = 32.6193
   Iteration :  13  (B r, r) = 30.5935
   Iteration :  14  (B r, r) = 26.8708
   Iteration :  15  (B r, r) = 25.1472
   Iteration :  16  (B r, r) = 23.2143
   Iteration :  17  (B r, r) = 20.548
   Iteration :  18  (B r, r) = 21.4497
   Iteration :  19  (B r, r) = 28.8504
   Iteration :  20  (B r, r) = 40.5788
   Iteration :  21  (B r, r) = 50.9547
   Iteration :  22  (B r, r) = 52.7938
   Iteration :  23  (B r, r) = 57.6496
  

Size of linear system: 8134
Average reduction factor = 0.866892
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.34
   Iteration :   2  (B r, r) = 625.219
   Iteration :   3  (B r, r) = 293.239
   Iteration :   4  (B r, r) = 191.884
   Iteration :   5  (B r, r) = 149.029
   Iteration :   6  (B r, r) = 116.207
   Iteration :   7  (B r, r) = 91.785
   Iteration :   8  (B r, r) = 71.5791
   Iteration :   9  (B r, r) = 55.1813
   Iteration :  10  (B r, r) = 42.4382
   Iteration :  11  (B r, r) = 36.3414
   Iteration :  12  (B r, r) = 32.6025
   Iteration :  13  (B r, r) = 30.574
   Iteration :  14  (B r, r) = 26.8499
   Iteration :  15  (B r, r) = 25.1239
   Iteration :  16  (B r, r) = 23.1889
   Iteration :  17  (B r, r) = 20.5218
   Iteration :  18  (B r, r) = 21.4183
   Iteration :  19  (B r, r) = 28.8033
   Iteration :  20  (B r, r) = 40.5075
   Iteration :  21  (B r, r) = 50.86
   Iteration :  22  (B r, r) = 52.6908
   Iteration :  23  (B r, r) = 57.5319
   I

Size of linear system: 8134
Average reduction factor = 0.86689
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.34
   Iteration :   2  (B r, r) = 625.219
   Iteration :   3  (B r, r) = 293.238
   Iteration :   4  (B r, r) = 191.883
   Iteration :   5  (B r, r) = 149.028
   Iteration :   6  (B r, r) = 116.206
   Iteration :   7  (B r, r) = 91.7837
   Iteration :   8  (B r, r) = 71.5776
   Iteration :   9  (B r, r) = 55.1798
   Iteration :  10  (B r, r) = 42.4366
   Iteration :  11  (B r, r) = 36.3395
   Iteration :  12  (B r, r) = 32.6004
   Iteration :  13  (B r, r) = 30.5716
   Iteration :  14  (B r, r) = 26.8473
   Iteration :  15  (B r, r) = 25.121
   Iteration :  16  (B r, r) = 23.1857
   Iteration :  17  (B r, r) = 20.5186
   Iteration :  18  (B r, r) = 21.4144
   Iteration :  19  (B r, r) = 28.7975
   Iteration :  20  (B r, r) = 40.4986
   Iteration :  21  (B r, r) = 50.8482
   Iteration :  22  (B r, r) = 52.678
   Iteration :  23  (B r, r) = 57.5172
   

Size of linear system: 8134
Average reduction factor = 0.86689
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.34
   Iteration :   2  (B r, r) = 625.219
   Iteration :   3  (B r, r) = 293.238
   Iteration :   4  (B r, r) = 191.883
   Iteration :   5  (B r, r) = 149.029
   Iteration :   6  (B r, r) = 116.206
   Iteration :   7  (B r, r) = 91.7839
   Iteration :   8  (B r, r) = 71.5778
   Iteration :   9  (B r, r) = 55.18
   Iteration :  10  (B r, r) = 42.4368
   Iteration :  11  (B r, r) = 36.3398
   Iteration :  12  (B r, r) = 32.6007
   Iteration :  13  (B r, r) = 30.5719
   Iteration :  14  (B r, r) = 26.8477
   Iteration :  15  (B r, r) = 25.1213
   Iteration :  16  (B r, r) = 23.1861
   Iteration :  17  (B r, r) = 20.519
   Iteration :  18  (B r, r) = 21.4149
   Iteration :  19  (B r, r) = 28.7982
   Iteration :  20  (B r, r) = 40.4997
   Iteration :  21  (B r, r) = 50.8497
   Iteration :  22  (B r, r) = 52.6796
   Iteration :  23  (B r, r) = 57.5191
   I

Size of linear system: 8134
Average reduction factor = 0.86689
   Iteration :   0  (B r, r) = 10840.4
   Iteration :   1  (B r, r) = 1716.34
   Iteration :   2  (B r, r) = 625.219
   Iteration :   3  (B r, r) = 293.238
   Iteration :   4  (B r, r) = 191.883
   Iteration :   5  (B r, r) = 149.029
   Iteration :   6  (B r, r) = 116.206
   Iteration :   7  (B r, r) = 91.7839
   Iteration :   8  (B r, r) = 71.5778
   Iteration :   9  (B r, r) = 55.18
   Iteration :  10  (B r, r) = 42.4368
   Iteration :  11  (B r, r) = 36.3398
   Iteration :  12  (B r, r) = 32.6008
   Iteration :  13  (B r, r) = 30.572
   Iteration :  14  (B r, r) = 26.8477
   Iteration :  15  (B r, r) = 25.1214
   Iteration :  16  (B r, r) = 23.1862
   Iteration :  17  (B r, r) = 20.5191
   Iteration :  18  (B r, r) = 21.415
   Iteration :  19  (B r, r) = 28.7984
   Iteration :  20  (B r, r) = 40.5
   Iteration :  21  (B r, r) = 50.85
   Iteration :  22  (B r, r) = 52.6799
   Iteration :  23  (B r, r) = 57.5194
   Iterati

NameError: name 'pd' is not defined